In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/myColab/AlgoTrader

Mounted at /content/drive
/content/drive/MyDrive/myColab/AlgoTrader


In [ ]:
# Demo code sample. Not indended for production use.

# See instructions for installing Requests module for Python
# https://requests.readthedocs.io/en/master/user/install/#install

import requests
import json
def execute(year, month):
  requestUrl = "https://api.nytimes.com/svc/archive/v1/"+ str(year) + '/' + str(month) + ".json?api-key=GbtcLpgAK0zu4345cgNDqprstz2MtMuj"
  requestHeaders = {
    "Accept": "application/json"
  }

  response = requests.get(requestUrl, headers=requestHeaders)

  res_json = response.json()

  with open("NYTimes/" + str(year) + '_'+ str(month) +".json", "w") as file:
      json.dump(res_json, file)



In [ ]:
year = 2015
for month in range(11, 13):
  execute(year, month)



In [ ]:
year = 2022
for month in range(1, 6):
  execute(year, month)

In [ ]:
#execute()
with open('test.json') as fin:
  res_json = json.load(fin)

In [ ]:
print(res_json['response']['docs'])

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [3]:
!pip3 install sentence_transformers
import nltk
nltk.download('punkt')

from sentence_transformers import SentenceTransformer, LoggingHandler
from sentence_transformers import models, util, datasets, evaluation, losses
from torch.utils.data import DataLoader
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
# Define your sentence transformer model using CLS pooling
model_name = 'bert-base-uncased'
word_embedding_model = models.Transformer(model_name)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), 'cls')
model = SentenceTransformer(modules=[word_embedding_model, pooling_model], device='cuda')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
def get_info(line):
    time = line['pub_date']
    headline = line['headline']
    headline = re.sub('[{}]'.format(punctuation_string),"",headline)
    headline = headline.replace('\u2019', '')
    headline = headline.replace('\u2018', '')
    year = time[:4]
    month = time[5:7]
    day = time[8:10]
    embedding = model.encode([headline], convert_to_tensor=True)
    return year+month+day, embedding[0]

In [11]:
import json
import string
import numpy as np
import torch
import re
punctuation_string = string.punctuation
with open("NYTimes_all.json", 'r') as fin:
    dataset = json.load(fin)
time, embedding = get_info(dataset[0])
dataset_by_day = []
i = 1
num = 1
#print(embedding)
while (i < len(dataset)):
    if i % 1000 == 0:
        print(i, len(dataset))
    line = dataset[i]
    new_time, new_embedding = get_info(line)
    if new_time == time:
        embedding = embedding + new_embedding
        num += 1
        #headline += (' ' + new_headline)
    else:
        #print(embedding/torch.tensor(num))
        embedding = embedding / torch.tensor(num)
        embedding = embedding.cpu().numpy().tolist()
        dataset_by_day.append({"pub_date" : time, "embedding": embedding})
        embedding = new_embedding
        time = new_time
        num = 1
    i += 1
with open("NYTimes_all_embedding.json", 'w') as fout:
    json.dump(dataset_by_day, fout)

1000 184361
2000 184361
3000 184361
4000 184361
5000 184361
6000 184361
7000 184361
8000 184361
9000 184361
10000 184361
11000 184361
12000 184361
13000 184361
14000 184361
15000 184361
16000 184361
17000 184361
18000 184361
19000 184361
20000 184361
21000 184361
22000 184361
23000 184361
24000 184361
25000 184361
26000 184361
27000 184361
28000 184361
29000 184361
30000 184361
31000 184361
32000 184361
33000 184361
34000 184361
35000 184361
36000 184361
37000 184361
38000 184361
39000 184361
40000 184361
41000 184361
42000 184361
43000 184361
44000 184361
45000 184361
46000 184361
47000 184361
48000 184361
49000 184361
50000 184361
51000 184361
52000 184361
53000 184361
54000 184361
55000 184361
56000 184361
57000 184361
58000 184361
59000 184361
60000 184361
61000 184361
62000 184361
63000 184361
64000 184361
65000 184361
66000 184361
67000 184361
68000 184361
69000 184361
70000 184361
71000 184361
72000 184361
73000 184361
74000 184361
75000 184361
76000 184361
77000 184361
78000 18

In [12]:
print(dataset_by_day[1]['embedding'])

[-0.414422869682312, 0.059360355138778687, -0.18376439809799194, -0.06801397353410721, -0.22407665848731995, -0.08340353518724442, 0.32927295565605164, 0.4564332962036133, -0.23491039872169495, -0.15924586355686188, -0.07225485146045685, -0.2840709090232849, 0.004996724892407656, 0.4439975917339325, -0.09239127486944199, -0.021996883675456047, -0.18801000714302063, 0.43088194727897644, 0.15177448093891144, -0.08578205108642578, 0.09188593178987503, -0.37544307112693787, 0.040315549820661545, -0.04165241867303848, 0.12602683901786804, -0.13936035335063934, -0.0156905185431242, -0.021162575110793114, -0.05686619132757187, -0.008120685815811157, -0.01858922280371189, 0.20779640972614288, -0.21635502576828003, -0.040092356503009796, 0.3457869291305542, -0.19715382158756256, 0.29723992943763733, -0.11032453179359436, -0.01501910388469696, 0.10485491901636124, -0.10748282074928284, -0.008359425701200962, 0.2732276916503906, 0.006690595299005508, 0.06434168666601181, -0.3442281186580658, -2.3